## Conditional Random Field Model (Zero Window)

In [10]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import re
from collections import OrderedDict

#Custom models
from prepro import readfile, get_sentence, is_number, extract_words,get_label,partial_tags

#Model
from sklearn_crfsuite import CRF

#Evalulation
from sklearn.model_selection import cross_val_predict
from sklearn_crfsuite.metrics import flat_classification_report
from sklearn_crfsuite.metrics import flat_f1_score

In [11]:
#import data from my github repo
#test = readfile("test.txt")
data =readfile("train.txt")
print("Number of Sentences in dataset:",len(data))

Number of Sentences in dataset: 8961


In [3]:
train,test=train_test_split(data,train_size=.8, random_state=0)

In [12]:
data[0]

[['Vertaling', 'OUT'],
 ['van', 'OUT'],
 ['die', 'OUT'],
 ['inligting', 'OUT'],
 ['in', 'OUT'],
 ['die', 'OUT'],
 ['oorblywende', 'OUT'],
 ['amptelike', 'OUT'],
 ['tale', 'OUT'],
 ['sal', 'OUT'],
 ['na', 'OUT'],
 ['verwagting', 'OUT'],
 ['teen', 'OUT'],
 ['Maart', 'B-MISC'],
 ['2007', 'I-MISC'],
 ['afgehandel', 'OUT'],
 ['wees', 'OUT'],
 ['.', 'OUT']]

## Orthographic Feature Engineering

In [4]:
#Special Features:
def count_vowel(word):
    '''
    Function returns the number of vowels in token
    '''
    return sum(list(map(word.lower().count, "aeiou")))

def dash(word):
    '''
    The Function returns whether or not the token contains a dash
    '''
    return 1 if "-" in word else 0

def count_consonants(word):
    '''
    The Function returns the number of consonants in a token
    '''
    vowels="aeiou"
    return sum(i not in vowels for i in word)

def contain_punct(word):
    '''
    The Function returns the boolean if punctuations is present in token
    '''
    if re.match(r'^\w+$',word):return False
    else: return True
    
def apostrophe(word):
    '''
    The Function returns boolean if "'s" is present in token 
    '''
    if word =="'s":return True
    else:return False
    
def word_pattern(word):
    '''
    The Function returns word patter feature
    Upper Case = "A"
    Lower Case = "a"
    Digit = "0"
    '''
    token=""
    for i in word:
        if i.isupper():
            token +="A"
        elif i.islower():
            token +="a"
        elif i.isdigit():
            token +="0"
        else:
            token +=str(i)
    return token

def pattern_sum(word):
    '''
    The Function returns the word patern without consectutive duplicates
    '''
    return ''.join(OrderedDict.fromkeys(word))
    

In [5]:
def word2features(sentence,i):
    word = sentence[i]

    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        
        'Prefix_2': word[:2], # prefix
        'Prefix_3': word[:3], # prefix
        'word.isupper()': word.isupper(), #all caps
        'word.islower()': word.islower(),# all lower case
        'word.istitle()': word.istitle(), #starts with caps
        'word.isdigit()': word.isdigit(), #all digits
        'word.isalpha()': word.isalpha(), #all letters
        'word.isalnum()': word.isalnum(), #mixture of letters and digits
        
        'word.pattern()': word_pattern(word),#word pattern
        'word.pattern_sum()': pattern_sum(word_pattern(word)),
        'word.punct()':contain_punct(word),#contains punctuation
        'word.apost()':apostrophe(word), #is an apostrophe
        
        'Suffix_2': word[-2:], # suffix
        'Suffix_3': word[-3:], # suffix
        
    }
    if i == 0:
        features['START'] = True
        
    if i == len(sentence)-1:
        features['END'] = True
    
    return features

def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def get_all_sentences(dataset):
    sentences=[]
    for i in range(len(dataset)):
        sentences.append(get_sentence(dataset,i+1))
    return sentences

def get_all_labels(dataset):
    labels=[]
    for i in range(len(dataset)):
        #labels.append(partial_tags(get_label(dataset,i+1)))
        labels.append(get_label(dataset,i+1))
    return labels

In [6]:
#get the first sentence
sent = get_sentence(train,1)
#label = partial_tags(get_label(train,1))
label = get_label(train,1)

#print out first sentence
print(sent)
print(label)

['.', '3', ',', '9', ',', '15', 'Minister', 'van', 'Omgewingsake', 'en', 'Toerisme', '.']
['OUT', 'OUT', 'OUT', 'OUT', 'OUT', 'OUT', 'B-PERS', 'I-PERS', 'I-PERS', 'I-PERS', 'I-PERS', 'OUT']


In [7]:
#Applt feature engineering
train_sents = get_all_sentences(train)
train_labels = get_all_labels(train)
test_sents = get_all_sentences(test)
test_labels = get_all_labels(test)

X_train = [sent2features(s) for s in train_sents]
y_train = train_labels
X_test = [sent2features(s) for s in test_sents]
y_test = test_labels


sub_labels=list(set([item for sublist in train_labels for item in sublist]))
sub_labels.remove("OUT")


## Condtional Random Field Model


In [8]:
%%time
crf4 = CRF(algorithm='lbfgs',
          c1=0.1,
          c2=0.2,
          max_iterations=50,
          all_possible_transitions=False)

#training model
crf4.fit(X=X_train, y=y_train)

#generate predictions
pred = crf4.predict(X_train)

#generate report on entire model
report = flat_classification_report(y_pred=pred, y_true=y_train,labels=sub_labels)
print(report)

score=flat_f1_score(y_pred=pred, y_true=y_train,average='micro',labels=sub_labels)
print("F1=%.2f"%(score*100))

              precision    recall  f1-score   support

       I-LOC       0.93      0.92      0.93       288
      I-PERS       0.94      0.95      0.94      1715
      I-MISC       0.89      0.90      0.90      5371
       I-ORG       0.90      0.93      0.92      2164
      B-PERS       0.95      0.93      0.94      1627
      B-MISC       0.93      0.86      0.89      5264
       B-LOC       0.96      0.94      0.95      1478
       B-ORG       0.89      0.93      0.91      2831

   micro avg       0.92      0.91      0.91     20738
   macro avg       0.92      0.92      0.92     20738
weighted avg       0.92      0.91      0.91     20738

F1=91.18
Wall time: 12.2 s


# Predict on Test dataset

In [9]:
#prediction with best performaning model
pred = crf4.predict(X_test)

#generate report on entire model
report = flat_classification_report(y_pred=pred, y_true=y_test,labels=sub_labels)
print(report)

score=flat_f1_score(y_pred=pred, y_true=y_test,average='micro',labels=sub_labels)
print("F1=%.2f"%(score*100))

              precision    recall  f1-score   support

       I-LOC       0.57      0.46      0.51        50
      I-PERS       0.76      0.81      0.79       440
      I-MISC       0.73      0.73      0.73      1357
       I-ORG       0.73      0.68      0.70       551
      B-PERS       0.83      0.78      0.81       412
      B-MISC       0.79      0.72      0.75      1267
       B-LOC       0.85      0.74      0.79       340
       B-ORG       0.78      0.77      0.77       726

   micro avg       0.77      0.74      0.75      5143
   macro avg       0.76      0.71      0.73      5143
weighted avg       0.77      0.74      0.75      5143

F1=75.19
